In [1]:
import json
from copy import deepcopy
from shapely.geometry import shape, Point

def load_geojson(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        geojson_data = json.load(file)
    for feature in geojson_data['features']:
        geometry = shape(feature['geometry'])
        centroid = geometry.centroid
        feature['properties']['lat'] = centroid.y
        feature['properties']['long'] = centroid.x
    return geojson_data

def clear_properties(geojson_data):
    for feature in geojson_data['features']:
        p = feature['properties']
        feature['properties'] = {
            "FID": p["FID"],
            "ID": p["ID"],
            "kabko": p["kabko"],
            "lat": p["lat"],
            "long": p["long"]
        }
    return geojson_data
        
def clear_features(geojson_data):
    geojson_data['features'] = []
    return geojson_data

def append_data(geojson_data, new_data, base_data=None):
    base_data = base_data or geojson_data
    for feature in base_data['features']:
        if new_data.kabko == feature['properties']['kabko']:
            new_feature = deepcopy(feature)
            new_feature["geometry"] = feature["geometry"]
            #new_feature['properties'].update(new_data)
            new_data._apply(new_feature['properties'])
            geojson_data['features'].append(new_feature)
            return
    raise Exception(f"Kabko not found: {new_data.kabko}")
    

def export_geojson(geojson_data, output_path):
    with open(output_path, 'w', encoding='utf-8') as file:
        json.dump(geojson_data, file, ensure_ascii=False, indent=4)

In [2]:
folder = "arcgis export/"
geojson_data = load_geojson(folder + "jatim2_test.geojson")
shape_layer = clear_properties(geojson_data)
real_layer = clear_features(deepcopy(shape_layer))
pred_layer = deepcopy(real_layer)

In [3]:
shape_layer["features"][0]["properties"]

{'FID': 1,
 'ID': 1,
 'kabko': 'KAB. BANGKALAN',
 'lat': -7.044992023914042,
 'long': 112.92884608988699}

In [4]:


real_layer_list = []
pred_layer_list = []


def append_data_2(list_data, new_data, base_data):
    for feature in base_data['features']:
        if new_data.kabko == feature['properties']['kabko']:
            new_feature = deepcopy(feature['properties'])
            new_data._apply(new_feature)
            list_data.append(new_feature)
            return
    raise Exception(f"Kabko not found: {new_data.kabko}")

In [5]:
import os
import sys
from dotenv import load_dotenv
load_dotenv()
from prediksicovidjatim import util, config, database
from prediksicovidjatim.data.model import ModelDataRepo

ARCGIS_USER = os.getenv("ARCGIS_USER")
ARCGIS_PASS = os.getenv("ARCGIS_PASS")
ARCGIS_PORTAL = os.getenv("ARCGIS_PORTAL")

REAL_LAYER_ID = os.getenv("REAL_LAYER_ID")
PRED_LAYER_ID = os.getenv("PRED_LAYER_ID")

PREDICT_DAYS = 30
try:
    PREDICT_DAYS = int(os.getenv("PREDICT_DAYS"))
except Exception:
    pass
    
FIRST_TANGGAL_STR = os.getenv("FIRST_TANGGAL") or '2020-03-20'
FIRST_TANGGAL = util.parse_date(FIRST_TANGGAL_STR)

def init(cur=None):
    database.init()
    #config.init_plot()
    with database.get_conn() as conn, conn.cursor() as cur:
        ModelDataRepo.init_weights(cur)
    global FIRST_TANGGAL
    FIRST_TANGGAL = MapDataRepo.get_oldest_tanggal(None, None)

In [6]:

from memory_profiler import profile as mprofile
#sys.path.insert(0, "../../")
from prediksicovidjatim.data.map import MapDataRepo, MapDataPred, MapDataReal
from prediksicovidjatim.modeling import SeicrdRlcModel
from prediksicovidjatim.mapping import MapUpdater
from prediksicovidjatim.util import ThreadPool, lprofile
from requests.exceptions import ConnectionError
from threading import RLock
import gc

model_lock = RLock()
def predict(kabko, predict_days=PREDICT_DAYS):
    mod = SeicrdRlcModel(kabko)
    params = kabko.get_params_init(extra_days=predict_days)
    
    model_lock.acquire()
    model_result = mod.model(**params)
    model_lock.release()
    
    pred_data = MapDataPred.from_result(kabko, model_result)
    pred_data = MapDataPred.shift(pred_data, FIRST_TANGGAL)
    return pred_data

    
def get_updater(chunk_size=100):
    return MapUpdater(ARCGIS_PORTAL, ARCGIS_USER, ARCGIS_PASS, chunk_size=chunk_size)
    
def _update_map(updater, layer, layer_list, selected_kabko, layer_data, update=True, chunk_size=100, max_process_count=None, max_tasks_per_child=10):
    print(layer_data[0].kabko)
    for data in layer_data:
        append_data(layer, data, shape_layer)
        append_data_2(layer_list, data, shape_layer)
    return len(layer_data), len(layer_data)

def update_map(selected_kabko, chunk_size=100, tanggal=None, predict_days=PREDICT_DAYS, update_prediction=False, update_real=False, updater=None, db=None, max_process_count=None, max_tasks_per_child=2):
    #print("Updating maps of %s" % (selected_kabko,))
    updater = updater or get_updater(chunk_size=chunk_size)
    
    db = db or database
    done = 0
    with db.get_conn() as conn, conn.cursor() as cur:
        real_data = MapDataRepo.fetch_real_data(selected_kabko, cur)
        kabko = MapDataRepo.get_kabko_full(selected_kabko, cur)
        
    real_data = MapDataReal.shift(real_data, FIRST_TANGGAL)
    done2, chunk_size2 = _update_map(updater, real_layer, real_layer_list, selected_kabko, real_data, update=update_real, chunk_size=chunk_size, max_process_count=max_process_count, max_tasks_per_child=max_tasks_per_child)
    #gc.collect()
    done += done2
    chunk_size = min(chunk_size, chunk_size2)
    
    if predict_days > 0:
        pred_data = predict(kabko, predict_days)
        done2, chunk_size2 = _update_map(updater, pred_layer, pred_layer_list, selected_kabko, pred_data, update=update_prediction, chunk_size=chunk_size, max_process_count=max_process_count, max_tasks_per_child=max_tasks_per_child)
        #gc.collect()
        done += done2
        chunk_size = min(chunk_size, chunk_size2)
    
    #MapDataRepo.set_updated(selected_kabko, tanggal, chunk_size)
    print("Updated maps of %s" % (selected_kabko,))
    
def cache_geometry(updater=None, first_tanggal=FIRST_TANGGAL_STR):
    updater = updater or get_updater()
    layer = updater.get_layer(REAL_LAYER_ID)
    updater.cache_kabko_geometry(layer, util.format_date(first_tanggal))
    
def update_map_all(any=True, predict_days=PREDICT_DAYS, update_prediction=True, update_real=False, max_process_count=None, max_tasks_per_child=10, pool=None, inner_max_process_count=1, inner_max_tasks_per_child=100):
    latest_tanggal = None
    with database.get_conn() as conn, conn.cursor() as cur:
        #latest_tanggal = MapDataRepo.get_latest_tanggal(cur)
        kabko = MapDataRepo.fetch_kabko_need_mapping(latest_tanggal, any=any, cur=cur)
        
    print("%s kabko needs updating" % (str(len(kabko)),))
            
    if len(kabko) == 0:
        print("No kabko to update maps")
        return
        
    print("Caching kabko geometry")
    #cache_geometry()
    
    print("Updating maps of %s kabko" % (str(len(kabko)),))
    updater = None#get_updater()
    
    args = [(*k, latest_tanggal, predict_days, update_prediction, update_real, updater, database.singleton, inner_max_process_count, inner_max_tasks_per_child) for k in kabko]
    
    if not pool and max_process_count == 1:
        for arg in args:
            update_map(*arg)
    else:
        #gc.collect()
        #pool = pool or Pool(processes=max_process_count, maxtasksperchild=max_tasks_per_child)
        pool = pool or ThreadPool(processes=util.min_none(len(args), max_process_count))
        try:
            output = pool.starmap(update_map, args)
            pool.close()
            pool.join()
            print("Done updating maps of %s kabko" % (len(args),))
        except ConnectionError as ex:
            raise
        finally:
            pool.terminate()
            del pool

C:\Users\Linearch\.conda\envs\prediksicovidjatim\lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [7]:

def map(any=False, predict_days=PREDICT_DAYS, update_prediction=True, update_real=False):
    init()
    update_map_all(any=any, predict_days=predict_days, update_prediction=update_prediction, update_real=update_real)
    
map(any=True)

38 kabko needs updating
Caching kabko geometry
Updating maps of 38 kabko
KAB. BANGKALAN
KAB. BANYUWANGI
KAB. BLITAR
KAB. BOJONEGORO
KAB. BONDOWOSO
KAB. GRESIK
KAB. JEMBER
KAB. JOMBANG
KAB. KEDIRI
KAB. LAMONGAN
KAB. BANGKALAN
KAB. MADIUN
KAB. LUMAJANG
KAB. MAGETAN
KAB. MALANG
KAB. NGANJUK
KAB. MOJOKERTO
KAB. PACITAN
KAB. NGAWI
KAB. PAMEKASAN
KAB. BONDOWOSO
KAB. PASURUAN
KAB. PONOROGO
KAB. PROBOLINGGO
KAB. SAMPANG
KAB. SIDOARJO
KAB. SITUBONDO
KAB. SUMENEP
KAB. TRENGGALEK
KAB. TUBAN
KAB. TULUNGAGUNG
KOTA BATU
KOTA BLITAR
KOTA KEDIRI
KOTA MADIUN
KOTA MALANG
KOTA MOJOKERTO
KOTA PASURUAN
KOTA PROBOLINGGO
KOTA SURABAYA
KAB. JOMBANG
KAB. BLITAR
KAB. BOJONEGORO
KAB. BANYUWANGI
KAB. KEDIRI
KAB. GRESIK
KOTA MADIUN
KAB. JEMBER
KAB. LAMONGAN
Updated maps of KAB. BANGKALAN
KOTA MOJOKERTO
KAB. PACITAN
KAB. MADIUN
KAB. LUMAJANG
KAB. MAGETAN
KAB. NGAWI
KOTA BLITAR
KAB. MOJOKERTO
KOTA KEDIRI
KAB. NGANJUK
KAB. PASURUAN
KAB. MALANG
KAB. PAMEKASAN
KAB. PROBOLINGGO
KAB. TULUNGAGUNG
KOTA PROBOLINGGO
KAB. SIT

In [8]:
import pandas as pd

def export_csv(layer, path):
    df = pd.DataFrame(layer)
    df.to_csv(path, index=False)

In [9]:
export_geojson(shape_layer, folder + "shape_layer.geojson")
export_csv(real_layer_list, folder + "real_layer.csv")
export_csv(pred_layer_list, folder + "pred_layer.csv")
#export_geojson(real_layer, folder + "real_layer.geojson")
#export_geojson(pred_layer, folder + "pred_layer.geojson")